In [1]:
import time

import gym
import gym_snake
from ml_tools import q_learning

import numpy as np
import matplotlib.pyplot as plt

from ml_tools.ui import UI

ui = UI()
ui.connect("./public")

'127.0.0.1:57329'

In [2]:
model_name = "01_model_game-trained"

# brain_name = "baseline"
# brain_name = "pre-trained/baseline"
brain_name = "pre-trained/all-trainable"

# brain_name = "fast-epsilon-decay/pre-trained/all-trainable"

load_pre_trained = True

In [3]:
from brain.v2.brain import Brain
from preprocessor.v1.preprocessor import Preprocessor

preprocessor = Preprocessor()
preprocessor.ui = ui
brain = Brain(learning_rate=0)
brain.ui = ui
brain.load_model(name=brain_name, model_name=model_name)

brain.model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 50, 50, 3)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 11, 11, 32)        6176      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 32)         0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 2, 2, 64)          32832     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 2, 2, 64)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 256)               0   

In [4]:
agent = q_learning.Agent(
  action_space=[0, 1, 2, 3],

  brain=brain, preprocessor=preprocessor,

  epsilon_start = 0,
  epsilon_stop = 0,
  epsilon_decay_num_steps = 250,
  
  discount_factor=0.97,
  
  replay_batch_size=32*2,
  model_batch_size=16,

  steps_to_train=20,
  steps_to_update_target=40,

  replay_memory_max_len=50_000,
  replay_memory_min_len= 2_000,
)
agent.training = False

env = gym.make("snake-v0")

In [ ]:
raw_observation = env.reset()
agent.reset(raw_observation)

while True:
  ui.video.update_stream("raw", raw_observation)

  speed = int(ui.control.get_value("speed") or 100)
  if speed != 100:
    time.sleep(1 / speed)

  agent.training = True if (ui.control.get_value("training") or "true") == "true" else False
  action = agent.act()
  raw_next_observation, reward, done, info = env.step(action)
  agent.step(reward, raw_next_observation)

  raw_observation = raw_next_observation
  if done:
    raw_observation = env.reset()
    agent.reset(raw_observation)

4/4 [==============================] - 0s 3ms/step - loss: 0.0070
